In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [10]:
# read a CSV into a dataframe (adjust filename as needed)
events_filename = './data/event.csv'  # or provide a full path

if not os.path.exists(events_filename):
    csv_files = [f for f in os.listdir('.') if f.lower().endswith('.csv')]
    if not csv_files:
        raise FileNotFoundError(f"{events_filename!r} not found and no CSV files in the current directory.")
    filename = csv_files[0]
    print(f"No file named 'data.csv' found. Using first CSV in cwd: {filename}")

events = pd.read_csv(events_filename)
print(f"Loaded {len(events)} rows and {len(events.columns)} columns from {events_filename}")
events.head()

Loaded 107738 rows and 14 columns from ./data/event.csv


/var/folders/_b/sl_t4k5539781f29qf723b080000gn/T/ipykernel_51619/2221732598.py:11: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  events = pd.read_csv(events_filename)


,type,inPlay,isStrike,isBall,pitcherName,catcherName,batterName,pitchCode,pitchType,velocity,coordX,coordY,runners,PAId
0,PITCH,False,False,True,徐若熙,蔣少宏,陳晨威,B,FF,145,71.11,-91.43,[],NaN
1,PITCH,True,True,False,徐若熙,蔣少宏,陳晨威,H,FF,147,54.86,42.67,"[{'type': 'PA', 'runnerName': '陳晨威', 'isOut': ...",NaN
2,PITCH,False,True,False,徐若熙,蔣少宏,林立,F,FF,145,54.86,22.35,[],NaN
3,PITCH,False,True,False,徐若熙,蔣少宏,林立,SW,SL,135,-2.03,-56.89,[],NaN
4,PITCH,False,False,True,徐若熙,蔣少宏,林立,B,SL,139,-34.54,-144.25,[],NaN


In [11]:
# read a CSV into a dataframe (adjust filename as needed)
pa_filename = './data/pa.csv'  # or provide a full path

if not os.path.exists(pa_filename):
    csv_files = [f for f in os.listdir('.') if f.lower().endswith('.csv')]
    if not csv_files:
        raise FileNotFoundError(f"{pa_filename!r} not found and no CSV files in the current directory.")
    filename = csv_files[0]
    print(f"No file named 'data.csv' found. Using first CSV in cwd: {filename}")



pas = pd.read_csv(pa_filename, usecols=['inning', 'batterName', 'batterHand', 'pitcherName', 'pitcherHand', 'catcherName', 'paRound', 'pitchCodes', 'result', 'hardness', 'bases'])
print(f"Loaded {len(pas)} rows and {len(pas.columns)} columns from {pa_filename}")
pas.head()


Loaded 27600 rows and 11 columns from ./data/pa.csv


,inning,batterName,batterHand,pitcherName,pitcherHand,catcherName,paRound,bases,pitchCodes,result,hardness
0,1,陳晨威,L,徐若熙,R,蔣少宏,1,0,"['B', 'H']",GO,M
1,1,林立,R,徐若熙,R,蔣少宏,1,0,"['F', 'SW', 'B', 'B', 'S']",SO,NaN
2,1,梁家榮,L,徐若熙,R,蔣少宏,1,0,"['S', 'F', 'B', 'H']",2B,H
3,1,廖健富,L,徐若熙,R,蔣少宏,1,2,"['B', 'H']",1B,H
4,2,朱育賢,L,徐若熙,R,蔣少宏,1,0,"['S', 'SW', 'H']",FO,M


# Filter out non-pitching events

In [12]:
events = events[events['type'] == 'PITCH']

# Pitchers and batters PA>50

In [13]:
batter_names = pas['batterName'].unique()
print(f"Found {len(batter_names)} unique batters")

batter_names_filtered = []  
for batter in batter_names:
    pas_batter = pas[pas['batterName'] == batter]
    if len(pas_batter) < 50:
        continue
    batter_names_filtered.append(batter)

print(f'Filtered to {len(batter_names_filtered)} batters with at least 50 PA each')

Found 173 unique batters
Filtered to 113 batters with at least 50 PA each


In [14]:
pitcher_names = pas['pitcherName'].unique()
print(f"Found {len(pitcher_names)} unique pitchers")

pitcher_names_filtered = []
for pitcher in pitcher_names:
    pas_pitcher = pas[pas['pitcherName'] == pitcher]
    if len(pas_pitcher) < 50:
        continue
    pitcher_names_filtered.append(pitcher)

print(f'Filtered to {len(pitcher_names_filtered)} pitchers with at least 50 PA each')

Found 168 unique pitchers
Filtered to 124 pitchers with at least 50 PA each


In [15]:
pa_filtered = pas[pas['batterName'].isin(batter_names_filtered) & pas['pitcherName'].isin(pitcher_names_filtered) ]
print(f"Filtered PA count: {len(pa_filtered)}")

events_filtered = events[events['batterName'].isin(batter_names_filtered) & events['pitcherName'].isin(pitcher_names_filtered)]
print(f"Filtered Event count: {len(events_filtered)}")

Filtered PA count: 26009
Filtered Event count: 97552


# Pair PA and Events

In [16]:
# Pair plate-appearances (PA) from `pas`/`pa_filtered` with sequences of pitch `events` in `events_filtered`.
# Algorithm:
#  - Walk through events_filtered in order, accumulate pitches into a current PA.
#  - End a PA when: row['inPlay'] is True OR the row['runners'] contains a dict with type == 'PA' (robust end marker)
#  - Also force-end if batter/pitcher changes between consecutive rows.
#  - Build a pa_events dataframe, assign a sequential pa_seq per (batterName, pitcherName),
#    and then align with pas (or pa_filtered) by the same (batterName, pitcherName, pa_seq).
#
# Note: This cell uses variables already present in the notebook (events_filtered, pas or pa_filtered).
#       It does not re-import modules.

def _row_has_pa_marker(runners, batter_name):
    # runners may be a list of dicts or something else; be defensive
    if not runners:
        return False
    try:
        for r in runners:
            if isinstance(r, dict) and r.get('type') == 'PA':
                # optionally check runnerName matches batter_name if present
                rn = r.get('runnerName')
                if rn is None or rn == batter_name:
                    return True
    except Exception:
        return False
    return False

# ensure chronological order
events_seq = events_filtered.sort_index()

pa_events = []
current = None

for idx, row in events_seq.iterrows():
    b = row['batterName']
    p = row['pitcherName']

    # start new PA if none
    if current is None:
        current = {
            'batterName': b,
            'pitcherName': p,
            'start_idx': idx,
            'events_idx': [idx],
            'pitchCodes': [row.get('pitchCode')],
            'pitchTypes': [row.get('pitchType')],
            'velocities': [row.get('velocity')],
            'inPlay_flags': [row.get('inPlay')],
            'last_row': row
        }
        # if the single row already ends PA
        if row.get('inPlay') or _row_has_pa_marker(row.get('runners'), b):
            current['end_idx'] = idx
            pa_events.append(current)
            current = None
        continue

    # if batter or pitcher changed, finalize previous and start new
    if b != current['batterName'] or p != current['pitcherName']:
        current['end_idx'] = current['events_idx'][-1]
        pa_events.append(current)
        current = {
            'batterName': b,
            'pitcherName': p,
            'start_idx': idx,
            'events_idx': [idx],
            'pitchCodes': [row.get('pitchCode')],
            'pitchTypes': [row.get('pitchType')],
            'velocities': [row.get('velocity')],
            'inPlay_flags': [row.get('inPlay')],
            'last_row': row
        }
        if row.get('inPlay') or _row_has_pa_marker(row.get('runners'), b):
            current['end_idx'] = idx
            pa_events.append(current)
            current = None
        continue

    # otherwise append to current
    current['events_idx'].append(idx)
    current['pitchCodes'].append(row.get('pitchCode'))
    current['pitchTypes'].append(row.get('pitchType'))
    current['velocities'].append(row.get('velocity'))
    current['inPlay_flags'].append(row.get('inPlay'))
    current['last_row'] = row

    # end conditions
    if row.get('inPlay') or _row_has_pa_marker(row.get('runners'), b):
        current['end_idx'] = idx
        pa_events.append(current)
        current = None

# finalize any remaining current PA
if current is not None:
    current['end_idx'] = current['events_idx'][-1]
    pa_events.append(current)
    current = None

# build dataframe of pa events
pa_events_df = pd.DataFrame([{
    'batterName': e['batterName'],
    'pitcherName': e['pitcherName'],
    'start_idx': e['start_idx'],
    'end_idx': e['end_idx'],
    'events_idx': e['events_idx'],
    'pitchCodes_events': e['pitchCodes'],
    'pitchTypes_events': e['pitchTypes'],
    'velocities_events': e['velocities'],
    'inPlay_flags': e['inPlay_flags'],
    'n_pitches': len(e['events_idx'])
} for e in pa_events])

# assign sequential PA index per (batter, pitcher) to allow pairing with pas
pa_events_df['pa_seq'] = pa_events_df.groupby(['batterName', 'pitcherName']).cumcount() + 1

# prepare pas (use pa_filtered if you prefer pairing only filtered PAs)
pas_seq = pa_filtered.copy()
pas_seq = pas_seq.sort_index()  # keep original order
pas_seq['pa_seq'] = pas_seq.groupby(['batterName', 'pitcherName']).cumcount() + 1

# merge on batterName, pitcherName, pa_seq
paired = pd.merge(pas_seq, pa_events_df,
                  on=['batterName', 'pitcherName', 'pa_seq'],
                  how='left',
                  suffixes=('_pa', '_events'))

# show some diagnostics
print(f"Built {len(pa_events_df)} PA event groups from {len(events_seq)} events")
print(f"pas has {len(pas_seq)} rows; merged result has {len(paired)} rows")
paired[['batterName','pitcherName','pa_seq','n_pitches','start_idx','end_idx','pitchCodes','pitchCodes_events']].head(10)

Built 25859 PA event groups from 97552 events
pas has 26009 rows; merged result has 26009 rows


,batterName,pitcherName,pa_seq,n_pitches,start_idx,end_idx,pitchCodes,pitchCodes_events
0,陳晨威,徐若熙,1,2.0,0.0,1.0,"['B', 'H']","[B, H]"
1,林立,徐若熙,1,5.0,2.0,6.0,"['F', 'SW', 'B', 'B', 'S']","[F, SW, B, B, S]"
2,梁家榮,徐若熙,1,4.0,7.0,10.0,"['S', 'F', 'B', 'H']","[S, F, B, H]"
3,廖健富,徐若熙,1,2.0,11.0,12.0,"['B', 'H']","[B, H]"
4,朱育賢,徐若熙,1,3.0,13.0,15.0,"['S', 'SW', 'H']","[S, SW, H]"
5,林承飛,徐若熙,1,6.0,16.0,21.0,"['SW', 'SW', 'B', 'F', 'F', 'S']","[SW, SW, B, F, F, S]"
6,嚴宏鈞,徐若熙,1,4.0,23.0,26.0,"['B', 'B', 'S', 'H']","[B, B, S, H]"
7,陳晨威,徐若熙,2,5.0,32.0,36.0,"['B', 'S', 'B', 'F', 'SW']","[B, S, B, F, SW]"
8,林立,徐若熙,2,3.0,37.0,39.0,"['SW', 'B', 'H']","[SW, B, H]"
9,梁家榮,徐若熙,2,2.0,40.0,41.0,"['B', 'H']","[B, H]"
